<a href="https://colab.research.google.com/github/thomasjv799/Beta-VAE-on-Animal-Face/blob/master/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
! pip install transformers datasets evaluate

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from transformers import create_optimizer
from transformers.keras_callbacks import KerasMetricCallback
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
from transformers import TFDistilBertForSequenceClassification
from transformers import TFDistilBertPreTrainedModel, TFDistilBertMainLayer
from transformers import TFMobileBertForSequenceClassification
# hgf_model = "google/mobilebert-uncased"
hgf_2 = "distilbert/distilbert-base-uncased"
# hgf_mob = "mobilebert_uncased"

In [2]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')  # Use 'weighted', 'micro', or 'macro' based on your needs.

    return {
        "accuracy": accuracy,
        "f1": f1
    }


In [3]:
ds = load_dataset("csv", data_files="output_5_classes_updated2.csv",split="train")
ds = ds.train_test_split(test_size=0.1)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(hgf_2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
tokenized_query = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/13011 [00:00<?, ? examples/s]

Map:   0%|          | 0/1446 [00:00<?, ? examples/s]

In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [7]:
accuracy = evaluate.load("accuracy")

In [8]:
id2label = {0: "vehicle", 1: "user_manual",2:"generic",3:"productivity",4:"entertainment"}
label2id = {"vehicle": 0, "user_manual": 1,"generic":2,"productivity":3,"entertainment":4}

In [9]:
batch_size = 16
num_epochs = 10
batches_per_epoch = len(tokenized_query["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [10]:
# model_1 = "mobilebert_uncased"
# model_2 = "distillbert-uncased"

In [12]:
base_model = TFAutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path = hgf_2,
    num_labels=5,
    id2label=id2label,
    label2id=label2id
)

# model = ClippedModel(base_model, clip_value=1.0)
# model.compile(optimizer=optimizer)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [75]:
import tensorflow as tf
from transformers import TFDistilBertPreTrainedModel
from transformers.models.distilbert.modeling_tf_distilbert import TFMultiHeadSelfAttention

class TFClampedMultiHeadSelfAttention(TFMultiHeadSelfAttention):
    def __init__(self, config, clamp_value, **kwargs):
        super().__init__(config, **kwargs)
        self.clamp_value = clamp_value

    def call(self, query, key, value, mask, head_mask=None, output_attentions=False, training=False):
        # Call original attention
        outputs = super().call(
            query, key, value, mask,
            head_mask=head_mask,
            output_attentions=output_attentions,
            training=training
        )

        # Apply clamping to the attention output
        if isinstance(outputs, tuple):
            clamped = tf.clip_by_value(outputs[0], -self.clamp_value, self.clamp_value)
            return (clamped,) + outputs[1:]
        return tf.clip_by_value(outputs, -self.clamp_value, self.clamp_value)

class TFClampedDistilBertModel(TFDistilBertPreTrainedModel):
    def __init__(self, base_model, clamp_value=1.0):
        super().__init__(base_model.config)
        self.clamp_value = clamp_value

        # Create new transformer with clamped attention
        self.base_model = tf.keras.models.clone_model(base_model)

        # Replace attention layers with clamped versions
        for layer in self.base_model.distilbert.transformer.layer:
            original = layer.attention
            # Create new clamped attention layer
            layer.attention = TFClampedMultiHeadSelfAttention(
                config=base_model.config,
                clamp_value=clamp_value,
                name=original.name
            )
            # Copy weights from original attention layer
            layer.attention.set_weights(original.get_weights())

    def call(self, inputs, training=False):
        # Handle both dictionary and direct inputs
        if isinstance(inputs, dict):
            outputs = self.base_model(
                input_ids=inputs.get('input_ids'),
                attention_mask=inputs.get('attention_mask'),
                training=training
            )
        else:
            outputs = self.base_model(inputs, training=training)
        return outputs

    def prepare_tf_dataset(self, dataset, shuffle=True, batch_size=16, collate_fn=None):
        return self.base_model.prepare_tf_dataset(
            dataset,
            shuffle=shuffle,
            batch_size=batch_size,
            collate_fn=collate_fn
        )

# Usage:
# clamped_model = TFClampedDistilBertModel(base_model, clamp_value=1.0)

In [76]:
clamped_model = TFClampedDistilBertModel(base_model, clamp_value=1.0)

In [77]:
clamped_model.compile(optimizer=optimizer)

In [78]:
tf_train_set = clamped_model.prepare_tf_dataset(
    tokenized_query["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = clamped_model.prepare_tf_dataset(
    tokenized_query["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)


In [79]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [80]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Number of epochs with no improvement after which training will stop
    mode='min',         # Stop when monitored value stops decreasing
    restore_best_weights=True,  # Restore model weights from the epoch with the best value
    min_delta=1e-4      # Minimum change to qualify as an improvement
)

In [81]:
callbacks = [metric_callback,early_stopping]

In [83]:
history = clamped_model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks,verbose=1)

Epoch 1/3
813/813 [==============================] - 60s 74ms/step - loss: -388.0922 - val_loss: -623.3708 - accuracy: 0.1936 - f1: 0.0628
Epoch 2/3
813/813 [==============================] - 62s 76ms/step - loss: -903.2974 - val_loss: -1197.9688 - accuracy: 0.1936 - f1: 0.0628
Epoch 3/3
813/813 [==============================] - 60s 73ms/step - loss: -1507.2650 - val_loss: -1819.3340 - accuracy: 0.1936 - f1: 0.0628


In [84]:
# @title Default title text
model.save_pretrained("distilbert_1epoch")

NameError: name 'model' is not defined

In [37]:
model